## Data Preparation Process ##

In [1]:
# ## install finrl library
%%capture
!pip install git+https://github.com/Soumadip-Saha/FinRL.git

In [2]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import datetime

from finrl.neo_finrl.preprocessor.yahoodownloader import YahooDownloader
from finrl.neo_finrl.preprocessor.preprocessors import FeatureEngineer

In [3]:
from google.colab import files
uploaded = files.upload()
import io
df1 = pd.read_csv(io.BytesIO(uploaded['ind_nifty50list.csv']))
df1.head()

Saving ind_nifty50list.csv to ind_nifty50list.csv


,Company Name,Industry,Symbol,Series,ISIN Code
0,Adani Ports and Special Economic Zone Ltd.,SERVICES,ADANIPORTS,EQ,INE742F01042
1,Asian Paints Ltd.,CONSUMER GOODS,ASIANPAINT,EQ,INE021A01026
2,Axis Bank Ltd.,FINANCIAL SERVICES,AXISBANK,EQ,INE238A01034
3,Bajaj Auto Ltd.,AUTOMOBILE,BAJAJ-AUTO,EQ,INE917I01010
4,Bajaj Finance Ltd.,FINANCIAL SERVICES,BAJFINANCE,EQ,INE296A01024


In [4]:
new_tics = pd.unique(df1['Symbol']).tolist()
new_tics = [tics+'.NS' for tics in new_tics]
new_tics.remove('COALINDIA.NS')
new_tics.remove('HDFCLIFE.NS')
new_tics.remove('SBILIFE.NS')
print(new_tics)

['ADANIPORTS.NS', 'ASIANPAINT.NS', 'AXISBANK.NS', 'BAJAJ-AUTO.NS', 'BAJFINANCE.NS', 'BAJAJFINSV.NS', 'BPCL.NS', 'BHARTIARTL.NS', 'BRITANNIA.NS', 'CIPLA.NS', 'DIVISLAB.NS', 'DRREDDY.NS', 'EICHERMOT.NS', 'GRASIM.NS', 'HCLTECH.NS', 'HDFCBANK.NS', 'HEROMOTOCO.NS', 'HINDALCO.NS', 'HINDUNILVR.NS', 'HDFC.NS', 'ICICIBANK.NS', 'ITC.NS', 'IOC.NS', 'INDUSINDBK.NS', 'INFY.NS', 'JSWSTEEL.NS', 'KOTAKBANK.NS', 'LT.NS', 'M&M.NS', 'MARUTI.NS', 'NTPC.NS', 'NESTLEIND.NS', 'ONGC.NS', 'POWERGRID.NS', 'RELIANCE.NS', 'SHREECEM.NS', 'SBIN.NS', 'SUNPHARMA.NS', 'TCS.NS', 'TATACONSUM.NS', 'TATAMOTORS.NS', 'TATASTEEL.NS', 'TECHM.NS', 'TITAN.NS', 'UPL.NS', 'ULTRACEMCO.NS', 'WIPRO.NS']


In [5]:
df = YahooDownloader(start_date = '2009-01-01',
                     end_date = '2021-07-06',
                     ticker_list = new_tics).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [6]:
df.drop(df[df.date == '2010-02-06'].index, inplace=True)
!rm -rf my_data
!mkdir my_data
df.tic.value_counts()

HINDALCO.NS      3081
INFY.NS          3081
DRREDDY.NS       3081
ONGC.NS          3081
SHREECEM.NS      3081
ULTRACEMCO.NS    3081
ICICIBANK.NS     3081
BAJFINANCE.NS    3081
POWERGRID.NS     3081
TATAMOTORS.NS    3081
INDUSINDBK.NS    3081
TATACONSUM.NS    3081
RELIANCE.NS      3081
JSWSTEEL.NS      3081
DIVISLAB.NS      3081
UPL.NS           3081
CIPLA.NS         3081
IOC.NS           3081
ITC.NS           3081
BPCL.NS          3081
LT.NS            3081
HCLTECH.NS       3081
MARUTI.NS        3081
NTPC.NS          3081
BHARTIARTL.NS    3081
BRITANNIA.NS     3081
SBIN.NS          3081
BAJAJFINSV.NS    3081
GRASIM.NS        3081
AXISBANK.NS      3081
NESTLEIND.NS     3081
TATASTEEL.NS     3081
HINDUNILVR.NS    3081
ADANIPORTS.NS    3081
HEROMOTOCO.NS    3081
WIPRO.NS         3081
HDFCBANK.NS      3081
EICHERMOT.NS     3081
KOTAKBANK.NS     3081
M&M.NS           3081
BAJAJ-AUTO.NS    3081
TCS.NS           3081
SUNPHARMA.NS     3081
TECHM.NS         3081
TITAN.NS         3081
ASIANPAINT

In [7]:
fe = FeatureEngineer(
                    use_technical_indicator=False,
                    use_turbulence=False,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Data clean finished!


In [8]:
for tic in new_tics:
  df_new = df[df.tic == tic]
  df_new.sort_values(['date'], ignore_index = True)
  df_new.reset_index(drop = True, inplace = True)
  df_new.rename(columns={"tic":"symbol"}, inplace = True)
  df_new.drop("day", axis = 1, inplace = True)
  path = "my_data/"+tic[:-3]+".csv"
  df_new.to_csv(path, index = False)
  df_new.head()

In [1]:
%%capture
import sys, site
from pathlib import Path
try:
    import qlib
except ImportError:
    # install qlib
    ! pip install --upgrade numpy
    ! pip install pyqlib
    # reload
    site.main()
scripts_dir = Path.cwd().parent.joinpath("scripts")
if not scripts_dir.joinpath("get_data.py").exists():
    # download get_data.py script
    scripts_dir = Path("~/tmp/qlib_code/scripts").expanduser().resolve()
    scripts_dir.mkdir(parents=True, exist_ok=True)
    import requests
    with requests.get("https://raw.githubusercontent.com/microsoft/qlib/main/scripts/get_data.py") as resp:
        with open(scripts_dir.joinpath("get_data.py"), "wb") as fp:
            fp.write(resp.content)
    with requests.get("https://raw.githubusercontent.com/microsoft/qlib/main/scripts/dump_bin.py") as resp:
        with open(scripts_dir.joinpath("dump_bin.py"), "wb") as fp:
            fp.write(resp.content)

## Restart The Runtime ##

In [10]:
!python  ~/tmp/qlib_code/scripts/dump_bin.py dump_all --csv_path  /content/my_data --qlib_dir ~/.qlib/qlib_data/my_data --include_fields open,close,high,low,volume

2021-12-30 12:43:50.074 | INFO     | __main__:_get_all_date:272 - start get all date......
100% 47/47 [00:01<00:00, 34.53it/s]
2021-12-30 12:43:51.437 | INFO     | __main__:_get_all_date:291 - end of get all date.

2021-12-30 12:43:51.438 | INFO     | __main__:_dump_calendars:294 - start dump calendars......
2021-12-30 12:43:51.458 | INFO     | __main__:_dump_calendars:297 - end of calendars dump.

2021-12-30 12:43:51.458 | INFO     | __main__:_dump_instruments:300 - start dump instruments......
2021-12-30 12:43:51.459 | INFO     | __main__:_dump_instruments:302 - end of instruments dump.

2021-12-30 12:43:51.459 | INFO     | __main__:_dump_features:305 - start dump features......
100% 47/47 [00:01<00:00, 28.77it/s]
2021-12-30 12:43:53.093 | INFO     | __main__:_dump_features:312 - end of features dump.



## Model Building Section ##

In [5]:
import qlib
import pandas as pd
from qlib.config import REG_US
from qlib.utils import exists_qlib_data, init_instance_by_config
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord
from qlib.utils import flatten_dict

In [3]:
REG_INDIA = "In"
dic ={
    REG_INDIA: {
        "trade_unit": 1,
        "limit_threshold": 0.05,
        "deal_price": "close",
    },
}

In [6]:
provider_uri = "~/.qlib/qlib_data/my_data"  # target_dir
if not exists_qlib_data(provider_uri):
    print(f"Qlib data is not found in {provider_uri}")
    sys.path.append(str(scripts_dir))
    from get_data import GetData
    GetData().qlib_data(target_dir=provider_uri, region=REG_US)
    #GetData().csv_data_cn(target_dir="~/kqlib/csv_data/cn_data")
qlib.init(provider_uri=provider_uri, region=REG_US)

[457:MainThread](2021-12-30 13:14:38,732) INFO - qlib.Initialization - [config.py:384] - default_conf: client.
[457:MainThread](2021-12-30 13:14:38,739) WARNING - qlib.Initialization - [config.py:410] - redis connection failed(host=127.0.0.1 port=6379), DiskExpressionCache and DiskDatasetCache will not be used!
[457:MainThread](2021-12-30 13:14:38,748) INFO - qlib.Initialization - [__init__.py:57] - qlib successfully initialized based on client settings.
[457:MainThread](2021-12-30 13:14:38,750) INFO - qlib.Initialization - [__init__.py:59] - data_path={'__DEFAULT_FREQ': PosixPath('/root/.qlib/qlib_data/my_data')}


In [9]:
market = "all"
benchmark = "ADANIPORTS"

In [10]:
###################################
# train model
###################################
data_handler_config = {
    "start_time": "2009-01-01",
    "end_time": "2021-07-05",
    "fit_start_time": "2009-01-01",
    "fit_end_time": "2015-12-31",
    "instruments": market,
}

task = {
    "model": {
        "class": "LGBModel",
        "module_path": "qlib.contrib.model.gbdt",
        "kwargs": {
            "loss": "mse",
            "colsample_bytree": 0.8879,
            "learning_rate": 0.0421,
            "subsample": 0.8789,
            "lambda_l1": 205.6999,
            "lambda_l2": 580.9768,
            "max_depth": 8,
            "num_leaves": 210,
            "num_threads": 20,
        },
    },
    "dataset": {
        "class": "DatasetH",
        "module_path": "qlib.data.dataset",
        "kwargs": {
            "handler": {
                "class": "Alpha158",
                "module_path": "qlib.contrib.data.handler",
                "kwargs": data_handler_config,
            },
            "segments": {
                "train": ("2009-01-01", "2015-12-31"),
                "valid": ("2016-01-01", "2017-12-31"),
                "test": ("2018-01-01", "2021-07-05"),
            },
        },
    },
}

# model initiaiton
model = init_instance_by_config(task["model"])
dataset = init_instance_by_config(task["dataset"])

# start exp to train model
with R.start(experiment_name="train_model"):
    R.log_params(**flatten_dict(task))
    model.fit(dataset)
    R.save_objects(trained_model=model)
    rid = R.get_recorder().id


[457:MainThread](2021-12-30 13:58:36,114) INFO - qlib.timer - [log.py:113] - Time cost: 27.464s | Loading data Done
[457:MainThread](2021-12-30 13:58:36,276) INFO - qlib.timer - [log.py:113] - Time cost: 0.135s | DropnaLabel Done
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
[457:MainThread](2021-12-30 13:58:41,751) INFO - qlib.timer - [log.py:113] - Time cost: 5.470s | CSZScoreNorm Done
[457:MainThread](2021-12-30 13:58:41,753) INFO - qlib.timer - [log.py:113] - Time cost: 5.637s | fit & process data Done
[457:MainThread](2021-12-30 13:58:41,756) INFO - qlib.timer - [log.py:113] - Time cost: 33.107s | Init data Done
[457:MainThread](2021-12-30 13:58:41,759) IN

Training until validation scores don't improve for 50 rounds.
[20]	train's l2: nan	valid's l2: 0.978713
[40]	train's l2: nan	valid's l2: 0.978706


[457:MainThread](2021-12-30 13:58:46,832) INFO - qlib.timer - [log.py:113] - Time cost: 0.000s | waiting `async_log` Done


Early stopping, best iteration is:
[1]	train's l2: nan	valid's l2: 0.978723
